In [1]:
import os
import tensorflow as tf
print(tf.__version__)
import numpy as np
import run_reconstructed_model

2022-12-04 10:28:46.863366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.10.0


In [2]:
# SPECIFY DIRECTORY OF EXPORTED MODEL

trained_saved_model_dir = "."

In [3]:
# Output dir for saved model with preprocessing
saved_model_with_preprocessing_dir = os.path.join(trained_saved_model_dir, "saved_model_with_preprocessing")


In [4]:
reconstructed_model = tf.saved_model.load(saved_model_with_preprocessing_dir)

2022-12-04 10:28:54.335386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
test_input_fp = os.path.join(trained_saved_model_dir, "test_inputs","test_input.npy")
loaded_input = np.load(test_input_fp)
test_input = tf.convert_to_tensor(loaded_input, dtype=tf.float32)

In [6]:
y = reconstructed_model(test_input)

In [7]:
print(y[0][0][0])
# EXPECTED: tf.Tensor(-1.6962206, shape=(), dtype=float32)
print(y[3][0][0])
# EXPECTED: tf.Tensor(-3.8082335, shape=(), dtype=float32)

tf.Tensor(-1.6962236, shape=(), dtype=float32)
tf.Tensor(-3.808238, shape=(), dtype=float32)


In [8]:
# try processing a file

test_filename = os.path.join(trained_saved_model_dir, "test_inputs","LEBI.20110509.030808.46.wav")

target_sr = 22050
clip_length_sec = 1
stride_sec = 1

taxonomy_dir = os.path.join(trained_saved_model_dir, "taxonomy")

species_list_fp= os.path.join(taxonomy_dir, "species_select_v5.txt")
group_list_fp  = os.path.join(taxonomy_dir, "groups_select_v5.txt")
family_list_fp = os.path.join(taxonomy_dir,"families_select_v5.txt")
order_list_fp  = os.path.join(taxonomy_dir,"orders_select_v5.txt")

taxonomy_fp = os.path.join(taxonomy_dir,"ebird_taxonomy.csv")
group_map_fp = os.path.join(taxonomy_dir,"groups_ebird_codes.csv")

# this has the species subset
test_config_dir = os.path.join(trained_saved_model_dir, "test_config")

test_config_fp = os.path.join(test_config_dir,"test_config.json")

# calibrators
calibrators_fp = os.path.join(test_config_dir,"calibrators_dict.obj")


In [11]:
detect_df = run_reconstructed_model.run_model_on_file(
                    reconstructed_model, #####
                      test_filename,
                        target_sr,
                        clip_length_sec,
                        stride_sec,
                        species_list_fp,
                        group_list_fp,  
                        family_list_fp, 
                        order_list_fp,
                        taxonomy_fp,
                        group_map_fp,
                        calibrators_fp=calibrators_fp,
                        test_config_fp=test_config_fp,
                        quiet=False,
                        stream=True # don't load whole file into memory at once
            )

loading taxonomy
using taxon subset
processing file: streaming from file
processed 24.0 s of audio in 0.6 s (37.2x)
predictions generated
doing calibration
merging taxonomic predictions
done


/Users/benjamin/opt/anaconda3/envs/tf-2.1/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator _SigmoidCalibration from version 1.0.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
detect_df

,start_sec,end_sec,filename,path,order,prob_order,family,prob_family,group,prob_group,species,prob_species,class,prob
0,5,6,LEBI.20110509.030808.46.wav,./test_inputs/LEBI.20110509.030808.46.wav,Pelecaniformes,0.972657,Ardeidae,0.974137,NaN,NaN,leabit,0.999182,leabit,0.999182
1,12,13,LEBI.20110509.030808.46.wav,./test_inputs/LEBI.20110509.030808.46.wav,Pelecaniformes,0.973123,Ardeidae,0.974991,NaN,NaN,leabit,0.999848,leabit,0.999848
2,20,21,LEBI.20110509.030808.46.wav,./test_inputs/LEBI.20110509.030808.46.wav,Pelecaniformes,0.863307,Ardeidae,0.836264,NaN,NaN,NaN,NaN,Ardeidae,0.836264


In [13]:
out_dir = os.path.join(trained_saved_model_dir,'test_outputs')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

run_reconstructed_model.save_detections_to_file(detect_df, 
                            test_filename,
                            out_dir)

run_reconstructed_model.save_raven_selection_table(detect_df,
                               test_filename,
                               out_dir)